## Water Extraction Forecasting - **Daily Frequency**

Walkthrough video explaining step by step how to reproduce the notebook (if desired):
https://www.loom.com/share/5b87cb54e4ee4362a88fafb309b635a5

The data includes the date-time from 2013 to 2018 in a daily frequency. 

The complete feature list is as follows:

 
* Date: Day - Month - Year format
* Q: Volumetric flowrate used during that day from the natural sources to provide the city with the demanded water [DESIRED FORECAST FEATURE] 
* DHIAv: Direct Horizontal Irradiance - Average
* DHIMax: Direct Horizontal Irradiance - Max 
* DNIAv: Direct Normal Irradiance - Average
* DNIMax: Direct Normal Irradiance - Max
* GHIAv: Global Horizontal Irradiance - Average
* GHIMax: Global Horizontal Irradiance - Max
* DPMin: Dew Point - Min
* DPAv: Dew Point - Average
* DPMax: Dew Point - Max
* WSMin: Wind Speed - Min
* WSAv: Wind Speed - Average
* WSMax: Wind Speed - Max
* RainMin: Rain - Min
* RainAv: Rain - Average
* RainMax: Rain - Max
* RHMin: Relative Humidity - Min
* RHAv: Relative Humidity - Average
* RHMax: Relative Humidity - Max
* Tmin: Temperature - Min
* TAv: Temperature - Average
* TMax: Temperature - Max
* PMin: Pressure - Min
* PAv: Pressure - Average
* PMax: Pressure - Max
* Weekday: Weekday (1: yes) 
* Weekend: Weekend (1: yes)
* Festive: Festive (free labor day) (1: yes)
* year: Year, goes from 2013 to 2018
* day_of_month: Month, goes from 1 to 31
* day_of_week: Goes from 1 to 7
* month: Month, goes from 1 to 12

Total data points: 2191
Total features: 32

Based on:
* https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/

In [0]:
#Basic libraries
from numpy import concatenate
from matplotlib import pyplot
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt

#Pandas
import pandas as pd
from pandas import read_csv
from pandas import DataFrame
from pandas import concat

#Sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

#Tensorflow
import tensorflow as tf

#Seaborn for graphs
import seaborn as sns 
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


##Connection of our Google Drive to the notebook



In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


####This root path is specific for OUR GOOGLE DRIVE folder. If you want to reproduce this notebook, change root_path to where datasets are stored. We will provide these datasets in our github. You can download them, and upload them to a folder in your google drive for testing. 

In [0]:
root_path = '/content/gdrive/Shared drives/AguaYDrenaje/Datasets'

In [0]:
dataset = read_csv(root_path+"/postQData2.csv", parse_dates=['date'], index_col='date')
dataset

FileNotFoundError: ignored

##LSTM Data Preparation

This involves framing the dataset as a supervised learning problem and normalizing the input variables.

### Important: We will frame the supervised learning problem as predicting the Q (volumetric flowrate) of the current day (t) given today's features AND yesterday's Q. 

In [0]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [0]:
#Change dataframe into array for faster processing
values = dataset.values

# ensure all data is float
values = values.astype('float32')

# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit(values)
scaled = scaler.transform(values)

# frame as supervised learning
reframed = series_to_supervised(scaled, 1, 1)

# drop columns we don't want to predict
reframed.drop(reframed.columns[[33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63]], axis=1, inplace=True)

print(reframed)

##Define and Fit the model

In this section, we will fit a LSTM on the multivariate input data.

First, we must split the prepared dataset into train and test sets. For this demonstration, we will fit the model on the first 5 years of data, then evaluate it on the remaining 1 year of data.

The example below splits the dataset into train and test sets, then splits the train and test sets into input and output variables. Finally, the inputs (X) are reshaped into the 3D format expected by LSTMs, namely [samples, timesteps, features].

### Split into train and test sets

In [0]:
# split into train and test sets
values = reframed.values
n_train_days = 365*5
train = values[:n_train_days, :]
test = values[n_train_days:, :]

# split into input and outputs
X_train, y_train = train[:, :-1], train[:, -1]
X_test, y_test = test[:, :-1], test[:, -1]

# reshape input to be 3D [samples, timesteps, features]
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

We will define the LSTM with **70 neurons** in the **first hidden layer** and **1 neuron** in the **output** layer for predicting Q. The input shape will be 1 time step with 32 features.

We will use the **Mean Absolute Error (MAE)** loss function and the efficient **Adam version of stochastic gradient descent**.

The model will be fit for **50 training epochs** with a **batch size of True**. Remember that the internal state of the LSTM in Keras is reset at the end of each batch, so an internal state that is a function of a number of days may be helpful (try testing this).

Finally, we keep track of both the training and test loss during training by setting the validation_data argument in the fit() function. At the end of the run both the training and test loss are plotted.

In [0]:
# design network
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(70, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(tf.keras.layers.Dense(1))
model.compile(loss='mae', optimizer='adam')

In [0]:
# fit network
history = model.fit(X_train, y_train, epochs=50, batch_size=True, validation_data=(X_test, y_test), verbose=False, shuffle=False)

# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
plt.xlabel("epochs")
plt.ylabel("loss")
pyplot.legend()
pyplot.show()

Running the example first creates a plot showing the train and test loss during training.

Interestingly, we can see that test loss is above training loss. This suggests that the model is NOT overfitting the data. 

The Train and test loss are printed at the end of each training epoch. At the end of the run, the final RMSE of the model on the test dataset is printed.

## Prediction & Reverse Scaling

In [0]:
# make a prediction
y_pred = model.predict(X_test)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[2]))

In [0]:
# invert scaling for forecast
y_pred_inv = concatenate((y_pred, X_test[:, 1:]), axis=1)
y_pred_inv = scaler.inverse_transform(y_pred_inv)
y_pred_inv = y_pred_inv[:,0]

In [0]:
# invert scaling for actual
y_test = y_test.reshape((len(y_test), 1))
y_inv = concatenate((y_test, X_test[:, 1:]), axis=1)
y_inv = scaler.inverse_transform(y_inv)
y_inv = y_inv[:,0]

## Evaluate the model

After the model is fit, we can forecast for the entire test dataset.

We combine the forecast with the test dataset and invert the scaling. We also invert scaling on the test dataset with the expected pollution numbers.

With forecasts and actual values in their original scale, we can then calculate an error score for the model. In this case, we calculate the Root Mean Squared Error (RMSE) that gives error in the same units as the variable itself.

In [0]:
# calculate RMSE
rmse = sqrt(mean_squared_error(y_inv, y_pred_inv))
print('Test RMSE: %.3f' % rmse)

## Q predicted & real comparison

In [0]:
#Let's get together predicted and real into a DataFrame for easier comparison
y_comparison = pd.DataFrame({"Q real": y_inv,
                            "Q forecast": y_pred_inv})
y_comparison.head()

## Plotting results

In [0]:
plt.figure(figsize=(14, 8))
plt.title("Q real vs Q forecast")
plt.plot(y_comparison["Q real"], label = "Q real")
plt.plot(y_comparison["Q forecast"], label = "Q forecast")
plt.xlabel("Time")
plt.ylabel("Q")
plt.legend()